In [1]:
import heapq
import numpy as np
import tensorflow as tf
import os
import pickle
os.environ['CUDA_VISIBLE_DEVICES']='1'

/usr/local/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class Generate_Model:
    def __init__(self):
        self.gener_W = []
        self.gener_B = []
        self.gener_W.append(np.random.uniform(-1,1,[input_dim,hidden1_dim]).astype(np.float16))
        self.gener_B.append(np.random.uniform(-1,1,[hidden1_dim]).astype(np.float16))
        self.gener_W.append(np.random.uniform(-1,1,[hidden1_dim,hidden2_dim]).astype(np.float16))
        self.gener_B.append(np.random.uniform(-1,1,[hidden2_dim]).astype(np.float16))
        self.gener_W.append(np.random.uniform(-1,1,[hidden2_dim,hidden3_dim]).astype(np.float16))
        self.gener_B.append(np.random.uniform(-1,1,[hidden3_dim]).astype(np.float16))
        self.gener_W.append(np.random.uniform(-1,1,[hidden3_dim,hidden4_dim]).astype(np.float16))
        self.gener_B.append(np.random.uniform(-1,1,[hidden4_dim]).astype(np.float16))
        self.gener_W.append(np.random.uniform(-1,1,[hidden4_dim,hidden5_dim]).astype(np.float16))
        self.gener_B.append(np.random.uniform(-1,1,[hidden5_dim]).astype(np.float16))
        self.gener_W.append(np.random.uniform(-1,1,[hidden5_dim,hidden6_dim]).astype(np.float16))
        self.gener_B.append(np.random.uniform(-1,1,[hidden6_dim]).astype(np.float16))
        self.gener_W.append(np.random.uniform(-1,1,[hidden6_dim,output_dim]).astype(np.float16))
        self.gener_B.append(np.random.uniform(-1,1,[output_dim]).astype(np.float16))
        self.obj_num = 5
    
    def set_model(self,gener_W,gener_B):
        self.gener_W = gener_W
        self.gener_B = gener_B
        
    def sigmoid(self,x):
        sigm = 1. / (1. + np.exp(-x))
        return sigm
    def relu(self,x):
        relu = x * (x>0)
        return relu
    def softmax_Class(self,x):
        return np.exp(x) / np.sum(np.exp(x), axis=3).reshape([-1,10,10,1])
    def softmax_Cell(self,x):
        return np.exp(x) / np.sum(np.exp(x), axis=1).reshape([-1,1])
    
    def run(self,data):
        self.gener_Layer_output = []
        self.gener_Layer_output.append(np.add(np.matmul(data,self.gener_W[0]),self.gener_B[0]))
        self.gener_Layer_output[-1] = self.sigmoid(self.gener_Layer_output[-1])
        for i in range(6):
            self.gener_Layer_output.append(np.add(np.matmul(self.gener_Layer_output[-1],self.gener_W[i+1]),self.gener_B[i+1]))
            self.gener_Layer_output[-1] = self.sigmoid(self.gener_Layer_output[-1])
        self.output_reshape = self.gener_Layer_output[-1].reshape(-1,10,10,8)
        self.output_BB = self.output_reshape[:,:,:,:2]
        self.output_Cell = self.softmax_Cell(self.output_reshape[:,:,:,3].reshape(-1,10*10))
        self.output_Class = self.softmax_Class(self.output_reshape[:,:,:,3:])
    def get_new_obj_XY_Class(self):
        for i in range(self.output_reshape.shape[0]):
            nlarg_Cell = heapq.nlargest(self.obj_num, range(len(self.output_Cell[i])), self.output_Cell[i].take)
            output_Class_MaxIndex = np.argmax(self.output_Class,axis=3).reshape([-1,10,10,1])

            tmp_obj_XY = self.output_BB[i].reshape(-1,10*10,2)[:,nlarg_Cell,:]
            tmp_obj_Class = output_Class_MaxIndex[0].reshape(-1,10*10,1)[:,nlarg_Cell,:]
            if i == 0:
                self.new_obj_XY = tmp_obj_XY
                self.new_obj_Class = tmp_obj_Class
            else:
                self.new_obj_XY = np.append(self.new_obj_XY,tmp_obj_XY,axis=0)
                self.new_obj_Class = np.append(self.new_obj_Class,tmp_obj_Class,axis=0)
    def get_new_obj_tmp_Yolo_output(self,yolo_trainY):
        for i in range(self.output_reshape.shape[0]):
            nlarg_Cell = heapq.nlargest(self.obj_num, range(len(self.output_Cell[i])), self.output_Cell[i].take)
            ten_digit = np.divide(nlarg_Cell,10).astype(int)
            digit = np.mod(nlarg_Cell,10)
            new_obj_Yolo_X = np.divide(digit,10) + np.divide(self.new_obj_XY[i,:,0],10)
            new_obj_Yolo_Y = np.divide(ten_digit,10) + np.divide(self.new_obj_XY[i,:,1],10)
            tmp_new_obj_tmp_Yolo_output = np.append(new_obj_Yolo_X.reshape(-1,self.obj_num,1),new_obj_Yolo_Y.reshape(-1,self.obj_num,1),axis=2)

            grid_x, grid_y = np.where(yolo_trainY[i,:,:,5] == 1)
            obj = np.sort(yolo_trainY[i,grid_x,grid_y,:],axis=0)
            obj = obj[self.new_obj_Class[i].reshape(1,-1)]
            tmp_new_obj_tmp_Yolo_output = np.append(obj[:,:,0].reshape([-1,self.obj_num,1]),tmp_new_obj_tmp_Yolo_output,axis=2)
            tmp_new_obj_tmp_Yolo_output = np.append(tmp_new_obj_tmp_Yolo_output,obj[:,:,3:],axis=2)


            if i == 0:
                self.new_obj_tmp_Yolo_output = tmp_new_obj_tmp_Yolo_output
            else:
                self.new_obj_tmp_Yolo_output = np.append(self.new_obj_tmp_Yolo_output,tmp_new_obj_tmp_Yolo_output,axis=0)
    def get_obj_img(self,yolo_trainX,yolo_trainY):
        for j in range(self.output_reshape.shape[0]):
            grid_x, grid_y = np.where(yolo_trainY[j,:,:,5] == 1)
            obj = np.sort(yolo_trainY[j,grid_x,grid_y,:],axis=0)
            img_space_obj_Y = obj[:,1:6] * 608
            x_left = (img_space_obj_Y[:,0] - 15).astype(int)
            x_right = (img_space_obj_Y[:,0] + 15).astype(int)
            y_down = (img_space_obj_Y[:,1] -15).astype(int)
            y_top = (img_space_obj_Y[:,1] + 15).astype(int)
            for i in range(y_down.shape[0]):
                if i == 0:
                    tmp_img_space_obj_X = yolo_trainX[j,y_down[i]:y_top[i],x_left[i]:x_right[i],:].reshape(-1,30,30,3)
                else:
                    tmp_img_space_obj_X = np.append(tmp_img_space_obj_X,\
                                yolo_trainX[j,y_down[i]:y_top[i],x_left[i]:x_right[i],:].reshape(-1,30,30,3),axis=0)
            if j == 0:
                self.obj_img = tmp_img_space_obj_X.reshape(1,self.obj_num,30,30,3)
            else:
                self.obj_img = np.append(self.obj_img,tmp_img_space_obj_X.reshape(1,self.obj_num,30,30,3),axis=0)
    def get_new_img(self):
        for j in range(self.output_reshape.shape[0]):
            tmp_img = np.zeros((608, 608, 3), np.uint8)
            tmp_img[:,:,0] = 120
            tmp_img[:,:,1] = 118
            tmp_img[:,:,2] = 125

            new_img_space_obj_Y = (self.new_obj_tmp_Yolo_output[j,:,1:6] * 608).astype(int).reshape(5,5)
            for i in range(new_img_space_obj_Y.shape[0]):
                Y_down = new_img_space_obj_Y[i,1]-15
                Y_top = new_img_space_obj_Y[i,1]+15
                if Y_down < 0:
                    Y_top = Y_top-Y_down
                    Y_down = 0
                    self.new_obj_tmp_Yolo_output[j,i,2] = ((self.new_obj_tmp_Yolo_output[j,i,2]*608).astype(int)+15)/608
                if Y_top > 607:
                    Y_down = Y_down-(Y_top-607)
                    Y_top = 607
                    self.new_obj_tmp_Yolo_output[j,i,2] = ((self.new_obj_tmp_Yolo_output[j,i,2]*608).astype(int)-15)/608
                X_left = new_img_space_obj_Y[i,0]-15
                X_right = new_img_space_obj_Y[i,0]+15
                if X_left < 0:
                    X_right = X_right-X_left
                    X_left = 0
                    self.new_obj_tmp_Yolo_output[j,i,1] = ((self.new_obj_tmp_Yolo_output[j,i,1]*608).astype(int)+15)/608
                if X_right > 607:
                    X_left = X_left-(X_right-607)
                    X_right = 607
                    self.new_obj_tmp_Yolo_output[j,i,1] = ((self.new_obj_tmp_Yolo_output[j,i,1]*608).astype(int)-15)/608
                tmp_img[Y_down:Y_top,\
                        X_left:X_right,:] \
                = self.obj_img[j,self.new_obj_Class[j,i,0]]
            if j == 0:
                self.new_img = tmp_img.reshape(-1,608,608,3)
            else:
                self.new_img = np.append(self.new_img,tmp_img.reshape(-1,608,608,3),axis=0)
    def get_new_obj_Yolo_output(self):
        self.new_obj_Yolo_output = np.ones([self.new_img.shape[0],10,10,6])
        self.new_obj_Yolo_output[:,:,:,5] = np.multiply(self.new_obj_Yolo_output[:,:,:,5],0)
        for i in range(train_dataX.shape[0]):
            x = np.multiply(self.new_obj_tmp_Yolo_output[i,:,1].reshape(-1),10).astype(int)
            y = np.multiply(self.new_obj_tmp_Yolo_output[i,:,2].reshape(-1),10).astype(int)
            self.new_obj_Yolo_output[i,x,y,:] = self.new_obj_tmp_Yolo_output[i]

In [3]:
class GA():
    def __init__(self,Yolo):
        self.Generation_num = 20
        self.Yolo = Yolo
    def set_gener_model(self):
        self.gener_model = []
        for i in range(self.Generation_num):
            self.gener_model.append(Generate_Model())
            
    def normalize(self,x):
        if np.min(x) == np.max(x):
            return x/np.max(x)
        else:
            return (x-np.min(x))/(np.max(x)-np.min(x))
    
    def softmax(self,x):
        return np.exp(x) / np.sum(np.exp(x))
    
    def float_to_bit(self,array):
        tmp_bit = np.zeros([array.shape[0],array.shape[1],16])
        tmp_array = array

        first_dim,second_dim = np.where(tmp_array[:,:]<0)
        tmp_bit[first_dim,second_dim,0] = 1
        tmp_array = np.abs(tmp_array)
        for i in range(15):
            tmp_array = tmp_array*2
            first_dim,second_dim = np.where(tmp_array>1)
            tmp_bit[first_dim,second_dim,i+1] = 1
            tmp_array = np.mod(tmp_array,1)
        return tmp_bit
    
    def bit_to_float(self ,array):
        tmp_float = np.zeros([array.shape[0],array.shape[1],1])
        tmp_array = array

        for i in range(15):
            first_dim,second_dim = np.where(tmp_array[:,:,i+1]==1)
            tmp_float[first_dim,second_dim,0] = tmp_float[first_dim,second_dim,0] + 1/np.power(2,i+1)

        first_dim,second_dim = np.where(tmp_array[:,:,0]==1)
        tmp_float[first_dim,second_dim,0] = tmp_float[first_dim,second_dim,0] * -1

        return tmp_float.reshape(array.shape[0],array.shape[1])

    def run_gener_model(self,genera_input,img,label):
        for i in range(self.Generation_num):
            self.gener_model[i].run(genera_input)
            self.gener_model[i].get_new_obj_XY_Class()
            self.gener_model[i].get_new_obj_tmp_Yolo_output(label)
            self.gener_model[i].get_obj_img(img,label)
            self.gener_model[i].get_new_img()
            self.gener_model[i].get_new_obj_Yolo_output()
    def calculate_Adaptability(self):
        Adaptability = []
        for i in range(self.Generation_num):
#             print(i)
            img = self.gener_model[i].new_img
            Yolo_ans = self.gener_model[i].new_obj_Yolo_output
            with tf.Session() as sess:
                Yolo_ans = Yolo_ans.reshape(-1,10,10,6)
                a = Yolo_ans[:,:,:,0]

                a = sess.run(tf.one_hot(a, 14))
                Yolo_ans = np.append(a,Yolo_ans[:,:,:,1:],axis=3).reshape(-1,10*10,19)
            self.Yolo.run(img.reshape(-1,608,608,3)/255,Yolo_ans.reshape(-1,10*10,19),batch_size)
            if np.isnan(self.Yolo.loss) == True:
                Adaptability.append(1)
            else:
                Adaptability.append(self.Yolo.loss)
        print('calculate_Adaptability_done')
        return Adaptability        
    
    def get_adaptability_probability(self):
        self.adaptability = self.calculate_Adaptability()
        self.adaptability_normalize = self.normalize(np.asarray(self.adaptability))
        self.adaptability_softmax = self.softmax(self.adaptability_normalize)
        tmp = []
        for i in range(self.adaptability_softmax.shape[0]):
            tmp.append(np.sum(self.adaptability_softmax[:i+1]))
        self.adaptability_probability = np.asarray(tmp)
    
    def run_gener_model_weight(self):
        self.new_gener_model_weight=[]
        for k in range(int(self.Generation_num/2)):
#             print(k)
            choose_random = np.random.rand(1)
            choose_gener = np.where(self.adaptability_probability >= choose_random)[0]
            father_model = self.gener_model[choose_gener[0]]
            choose_random = np.random.rand(1)
            choose_gener = np.where(self.adaptability_probability >= choose_random)[0]
            mother_model = self.gener_model[choose_gener[0]]

            new1_gener_W = []
            new2_gener_W = []
            for i in range(len(father_model.gener_W)):
                father_generW_bit = self.float_to_bit(father_model.gener_W[i])
                mother_generW_bit = self.float_to_bit(mother_model.gener_W[i])

                father_generW_bit = father_generW_bit.reshape(father_generW_bit.shape[0],-1)
                mother_generW_bit = mother_generW_bit.reshape(mother_generW_bit.shape[0],-1)

                tmp1_gener_W = np.zeros(father_generW_bit.shape)
                tmp2_gener_W = np.zeros(father_generW_bit.shape)

                choose_mating_index = np.random.randint(1,mother_generW_bit.shape[1]-1)
                father_head =  father_generW_bit[:,:choose_mating_index]
                father_tail =  father_generW_bit[:,choose_mating_index:]

                mother_head =  mother_generW_bit[:,:choose_mating_index]
                mother_tail =  mother_generW_bit[:,choose_mating_index:]
                tmp1_gener_W[:,:] = np.append(father_head,mother_tail,axis=1)
                tmp2_gener_W[:,:] = np.append(mother_head,father_tail,axis=1)

                tmp1_gener_W = tmp1_gener_W.reshape(father_model.gener_W[i].shape[0],father_model.gener_W[i].shape[1],16)
                tmp2_gener_W = tmp2_gener_W.reshape(father_model.gener_W[i].shape[0],father_model.gener_W[i].shape[1],16)

                mutation_probability = np.random.rand(1) 
                if mutation_probability < 0.2:
                    mutation_index =  np.random.randint(1,16)
                    zero_first_dim,zero_second_dim = np.where(tmp1_gener_W[:,:,mutation_index] == 0.)
                    one_first_dim,one_second_dim = np.where(tmp1_gener_W[:,:,mutation_index] == 1.)

                    tmp1_gener_W[zero_first_dim,zero_second_dim,mutation_index] = 1.
                    tmp1_gener_W[one_first_dim,one_second_dim,mutation_index] = 0.

                    zero_first_dim,zero_second_dim = np.where(tmp2_gener_W[:,:,mutation_index] == 0.)
                    one_first_dim,one_second_dim = np.where(tmp2_gener_W[:,:,mutation_index] == 1.)
                    tmp2_gener_W[zero_first_dim,zero_second_dim,mutation_index] = 1.
                    tmp2_gener_W[one_first_dim,one_second_dim,mutation_index] = 0.

                new1_gener_W.append(self.bit_to_float(tmp1_gener_W))
                new2_gener_W.append(self.bit_to_float(tmp2_gener_W))
            self.new_gener_model_weight.append(new1_gener_W)
            self.new_gener_model_weight.append(new2_gener_W)
        print('run_gener_model_weight_done')
    def run_gener_model_bias(self):
        self.new_gener_model_bias=[]
        for k in range(int(self.Generation_num/2)):
#             print(k)
            choose_random = np.random.rand(1)
            choose_gener = np.where(self.adaptability_probability >= choose_random)[0]
            father_model = self.gener_model[choose_gener[0]]
            choose_random = np.random.rand(1)
            choose_gener = np.where(self.adaptability_probability >= choose_random)[0]
            mother_model = self.gener_model[choose_gener[0]]

            new1_gener_B = []
            new2_gener_B = []
            for i in range(len(father_model.gener_W)):
                father_generB_bit = self.float_to_bit(father_model.gener_B[i].reshape(1,-1))
                mother_generB_bit = self.float_to_bit(mother_model.gener_B[i].reshape(1,-1))

                father_generB_bit = father_generB_bit.reshape(father_generB_bit.shape[0],-1)
                mother_generB_bit = father_generB_bit.reshape(father_generB_bit.shape[0],-1)

                tmp1_gener_B = np.zeros(father_generB_bit.shape)
                tmp2_gener_B = np.zeros(father_generB_bit.shape)

                choose_mating_index = np.random.randint(1,mother_generB_bit.shape[1]-1)
                father_head =  father_generB_bit[:,:choose_mating_index]
                father_tail =  father_generB_bit[:,choose_mating_index:]

                mother_head =  mother_generB_bit[:,:choose_mating_index]
                mother_tail =  mother_generB_bit[:,choose_mating_index:]
                tmp1_gener_B[:,:] = np.append(father_head,mother_tail,axis=1)
                tmp2_gener_B[:,:] = np.append(mother_head,father_tail,axis=1)

                tmp1_gener_B = tmp1_gener_B.reshape(1,father_model.gener_B[i].shape[0],16)
                tmp2_gener_B = tmp2_gener_B.reshape(1,father_model.gener_B[i].shape[0],16)

                mutation_probability = np.random.rand(1) 
                if mutation_probability < 0.2:
                    mutation_index =  np.random.randint(1,16)
                    zero_first_dim,zero_second_dim = np.where(tmp1_gener_B[:,:,mutation_index] == 0.)
                    one_first_dim,one_second_dim = np.where(tmp1_gener_B[:,:,mutation_index] == 1.)

                    tmp1_gener_B[0,zero_second_dim,mutation_index] = 1.
                    tmp1_gener_B[0,one_second_dim,mutation_index] = 0.

                    zero_first_dim,zero_second_dim = np.where(tmp2_gener_B[:,:,mutation_index] == 0.)
                    one_first_dim,one_second_dim = np.where(tmp2_gener_B[:,:,mutation_index] == 1.)
                    tmp2_gener_B[0,zero_second_dim,mutation_index] = 1.
                    tmp2_gener_B[0,one_second_dim,mutation_index] = 0.

                new1_gener_B.append(self.bit_to_float(tmp1_gener_B).reshape(-1))
                new2_gener_B.append(self. bit_to_float(tmp1_gener_B).reshape(-1))
            self.new_gener_model_bias.append(new1_gener_B)
            self.new_gener_model_bias.append(new2_gener_B)
        print('run_gener_model_bias_done')
    
    def change_mode(self):
        for i in range(self.Generation_num):
            self.gener_model[i].set_model(self.new_gener_model_weight[i],self.new_gener_model_bias[i])
    
    def get_target_model(self):
        adaptability = self.calculate_Adaptability()
        adaptability = np.asarray(adaptability)
        model_index = np.argmax(adaptability)
        print('best_generator_index ',model_index)
        return self.gener_model[model_index]

In [4]:
class YoloModel():
    def __init__(self, loc):
        # Create local graph and use it in the session
        self.graph = tf.Graph()
        self.sess = tf.Session(graph=self.graph)
        with self.graph.as_default():
            # Import saved model from location 'loc' into local graph
            self.saver = tf.train.import_meta_graph(loc + '.meta',
                                               clear_devices=True)
            self.saver.restore(self.sess, loc)      
            self.activation_loss = tf.get_collection('loss')[0]
            self.activation_pre = tf.get_collection('pre')[0]
            self.activation_optimize = tf.get_collection('optimizer')[0]
            self.activation_noNormPre = tf.get_collection('noNorm_pre')[0]
    def run(self, x,y,batch_size,learning_rate=0.01):
        # The 'x' corresponds to name of input placeholder
        batch_num = int(x.shape[0]/ batch_size)
        self.loss = 0
        for i in range(batch_num):
        
            self.loss = self.loss + self.sess.run(self.activation_loss, \
                                          feed_dict={"x:0": x[batch_size*i:batch_size*(i+1)] ,\
                                                     'y:0':y [batch_size*i:batch_size*(i+1)] ,\
                                                     'learning_rate:0':learning_rate})
        self.loss = self.loss/batch_num
    def pre_obj(self, x,y,learning_rate=0.01):
        # The 'x' corresponds to name of input placeholder
        self.pre =  self.sess.run(self.activation_noNormPre,\
                                    feed_dict={"x:0": x ,'y:0':y ,'learning_rate:0':learning_rate})
        
        self.loss = self.sess.run(self.activation_noNormPre, \
                                      feed_dict={"x:0": x ,'y:0':y ,'learning_rate:0':learning_rate})
    
    def optimize(self,dataX,dataY,learning_rate,training_epochs,batch_size):
        
        batch_num = int(dataX.shape[0]/ batch_size)
        training_epochs = int(training_epochs * 2 / batch_num)
        rate = int(training_epochs/10)
        print('batch_num : ',batch_num)
        for epoch in range(training_epochs):
            if epoch % rate == 0 and epoch != 0:
                learning_rate = learning_rate / 2
                print('epoch : ',epoch)
                print('batch_loss',batch_loss / batch_num)
        
#             print('epoch : ',epoch)
            batch_loss = 0

            for i in range(batch_num):
                batch_loss = batch_loss + self.sess.run(self.activation_loss, \
                                      feed_dict={"x:0":dataX[batch_size*i:batch_size*(i+1)] ,\
                                                 'y:0':dataY[batch_size*i:batch_size*(i+1)] ,\
                                                 'learning_rate:0':learning_rate})

                self.sess.run(self.activation_optimize,\
                                      feed_dict={"x:0":dataX[batch_size*i:batch_size*(i+1)] ,\
                                                 'y:0':dataY[batch_size*i:batch_size*(i+1)] ,\
                                                 'learning_rate:0':learning_rate})
                
#             print('batch_loss',batch_loss / batch_num)
        
        
    def save(self,loc):
        self.saver.save(self.sess, loc)

In [5]:
class LBIG:
    def __init__(self):
        self.gener_W = []
        self.gener_B = []
        self.gener_W.append(np.random.uniform(-1,1,[input_dim,hidden1_dim]).astype(np.float16))
        self.gener_B.append(np.random.uniform(-1,1,[hidden1_dim]).astype(np.float16))
        self.gener_W.append(np.random.uniform(-1,1,[hidden1_dim,hidden2_dim]).astype(np.float16))
        self.gener_B.append(np.random.uniform(-1,1,[hidden2_dim]).astype(np.float16))
        self.gener_W.append(np.random.uniform(-1,1,[hidden2_dim,hidden3_dim]).astype(np.float16))
        self.gener_B.append(np.random.uniform(-1,1,[hidden3_dim]).astype(np.float16))
        self.gener_W.append(np.random.uniform(-1,1,[hidden3_dim,hidden4_dim]).astype(np.float16))
        self.gener_B.append(np.random.uniform(-1,1,[hidden4_dim]).astype(np.float16))
        self.gener_W.append(np.random.uniform(-1,1,[hidden4_dim,hidden5_dim]).astype(np.float16))
        self.gener_B.append(np.random.uniform(-1,1,[hidden5_dim]).astype(np.float16))
        self.gener_W.append(np.random.uniform(-1,1,[hidden5_dim,hidden6_dim]).astype(np.float16))
        self.gener_B.append(np.random.uniform(-1,1,[hidden6_dim]).astype(np.float16))
        self.gener_W.append(np.random.uniform(-1,1,[hidden6_dim,output_dim]).astype(np.float16))
        self.gener_B.append(np.random.uniform(-1,1,[output_dim]).astype(np.float16))
        self.obj_num = 5
        
    def set_model(self,gener_W,gener_B):
        self.gener_W = gener_W
        self.gener_B = gener_B
        
    def sigmoid(self,x):
        sigm = 1. / (1. + np.exp(-x))
        return sigm
    def relu(self,x):
        relu = x * (x>0)
        return relu
    def softmax_Class(self,x):
        return np.exp(x) / np.sum(np.exp(x), axis=3).reshape([-1,10,10,1])
    def softmax_Cell(self,x):
        return np.exp(x) / np.sum(np.exp(x), axis=1).reshape([-1,1])
    
    
    def run(self,data):
        self.gener_Layer_output = []
        self.gener_Layer_output.append(np.add(np.matmul(data,self.gener_W[0]),self.gener_B[0]))
        self.gener_Layer_output[-1] = self.sigmoid(self.gener_Layer_output[-1])
        for i in range(6):
            self.gener_Layer_output.append(np.add(np.matmul(self.gener_Layer_output[-1],self.gener_W[i+1]),self.gener_B[i+1]))
            self.gener_Layer_output[-1] = self.sigmoid(self.gener_Layer_output[-1])
        self.output_reshape = self.gener_Layer_output[-1].reshape(-1,10,10,8)
        self.output_BB = self.output_reshape[:,:,:,:2]
        self.output_Cell = self.softmax_Cell(self.output_reshape[:,:,:,3].reshape(-1,10*10))
        self.output_Class = self.softmax_Class(self.output_reshape[:,:,:,3:])
    def get_new_obj_XY_Class(self,density):
        for i in range(self.output_reshape.shape[0]):
            nlarg_Cell = heapq.nlargest(self.obj_num*density, range(len(self.output_Cell[i])), self.output_Cell[i].take)
            output_Class_MaxIndex = np.argmax(self.output_Class,axis=3).reshape([-1,10,10,1])

            tmp_obj_XY = self.output_BB[i].reshape(-1,10*10,2)[:,nlarg_Cell,:]
            tmp_obj_Class = output_Class_MaxIndex[0].reshape(-1,10*10,1)[:,nlarg_Cell,:]
            if i == 0:
                self.new_obj_XY = tmp_obj_XY
                self.new_obj_Class = tmp_obj_Class
            else:
                self.new_obj_XY = np.append(self.new_obj_XY,tmp_obj_XY,axis=0)
                self.new_obj_Class = np.append(self.new_obj_Class,tmp_obj_Class,axis=0)
    def get_new_obj_tmp_Yolo_output(self,yolo_trainY,density):
        for i in range(self.output_reshape.shape[0]):
            nlarg_Cell = heapq.nlargest(self.obj_num*density, range(len(self.output_Cell[i])), self.output_Cell[i].take)
            ten_digit = np.divide(nlarg_Cell,10).astype(int)
            digit = np.mod(nlarg_Cell,10)
            new_obj_Yolo_X = np.divide(digit,10) + np.divide(self.new_obj_XY[i,:,0],10)
            new_obj_Yolo_Y = np.divide(ten_digit,10) + np.divide(self.new_obj_XY[i,:,1],10)
            tmp_new_obj_tmp_Yolo_output = np.append(new_obj_Yolo_X.reshape(-1,self.obj_num*density,1),new_obj_Yolo_Y.reshape(-1,5*density,1),axis=2)
            
            for density_time in range(density):
                grid_x, grid_y = np.where(yolo_trainY[i,density_time,:,:,5] == 1)
                if density_time == 0:
                    obj = np.sort(yolo_trainY[i,density_time,grid_x,grid_y,:],axis=0)
                else:
                    obj = np.append(obj,np.sort(yolo_trainY[i,density_time,grid_x,grid_y,:],axis=0),axis=0)
            self.index = np.random.randint(density,size=(self.obj_num*density,1))
            obj = obj[(self.new_obj_Class[i]+self.index*self.obj_num).reshape(-1)].reshape(1,self.obj_num*density,6)
            tmp_new_obj_tmp_Yolo_output = np.append(obj[:,:,0].reshape([-1,self.obj_num*density,1]),tmp_new_obj_tmp_Yolo_output,axis=2)
            tmp_new_obj_tmp_Yolo_output = np.append(tmp_new_obj_tmp_Yolo_output,obj[:,:,3:].reshape([-1,self.obj_num*density,3]),axis=2)
            if i == 0:
                self.new_obj_tmp_Yolo_output = tmp_new_obj_tmp_Yolo_output
            else:
                self.new_obj_tmp_Yolo_output = np.append(self.new_obj_tmp_Yolo_output,tmp_new_obj_tmp_Yolo_output,axis=0)
    def get_obj_img(self,yolo_trainX,yolo_trainY,density):
        for j in range(self.output_reshape.shape[0]):
            for density_time in range(density):
                grid_x, grid_y = np.where(yolo_trainY[j,density_time,:,:,5] == 1)
                obj = np.sort(yolo_trainY[j,density_time,grid_x,grid_y,:],axis=0)                
                img_space_obj_Y = obj[:,1:6] * 608
                x_left = (img_space_obj_Y[:,0] - 15).astype(int)
                x_right = (img_space_obj_Y[:,0] + 15).astype(int)
                y_down = (img_space_obj_Y[:,1] -15).astype(int)
                y_top = (img_space_obj_Y[:,1] + 15).astype(int)
                for i in range(y_down.shape[0]):
                    if i == 0 and density_time == 0:
                        tmp_img_space_obj_X = yolo_trainX[j,density_time,y_down[i]:y_top[i],x_left[i]:x_right[i],:].reshape(-1,30,30,3)
                    else:
                        tmp_img_space_obj_X = np.append(tmp_img_space_obj_X,\
                                    yolo_trainX[j,density_time,y_down[i]:y_top[i],x_left[i]:x_right[i],:].reshape(-1,30,30,3),axis=0)
            if j == 0:
                self.obj_img = tmp_img_space_obj_X.reshape(1,density*self.obj_num,30,30,3)
            else:
                self.obj_img = np.append(self.obj_img,tmp_img_space_obj_X.reshape(1,density*self.obj_num,30,30,3),axis=0)
    def get_new_img(self,density):
        for j in range(self.output_reshape.shape[0]):
            tmp_img = np.zeros((608, 608, 3), np.uint8)
            tmp_img[:,:,0] = 120
            tmp_img[:,:,1] = 118
            tmp_img[:,:,2] = 125

            new_img_space_obj_Y = (self.new_obj_tmp_Yolo_output[j,:,1:6] * 608)\
                                                    .astype(int).reshape(self.obj_num*density,5)
            
            for density_time in range(density):
                for i in range(int(new_img_space_obj_Y.shape[0]/density)):
                    Y_down = new_img_space_obj_Y[i+density_time*self.obj_num,1]-15
                    Y_top = new_img_space_obj_Y[i+density_time*self.obj_num,1]+15
                    if Y_down < 0:
                        Y_top = Y_top-Y_down
                        Y_down = 0
                        self.new_obj_tmp_Yolo_output[j,i+density_time*self.obj_num,2] = \
                                ((self.new_obj_tmp_Yolo_output[j,i+density_time*self.obj_num,2]*608).astype(int)+15)/608
                    if Y_top > 607:
                        Y_down = Y_down-(Y_top-607)
                        Y_top = 607
                        self.new_obj_tmp_Yolo_output[j,i+density_time*self.obj_num,2] = \
                                ((self.new_obj_tmp_Yolo_output[j,i+density_time*self.obj_num,2]*608).astype(int)-15)/608
                    
                    X_left = new_img_space_obj_Y[i+density_time*self.obj_num,0]-15
                    X_right = new_img_space_obj_Y[i+density_time*self.obj_num,0]+15
                    if X_left < 0:
                        X_right = X_right-X_left
                        X_left = 0
                        self.new_obj_tmp_Yolo_output[j,i+density_time*self.obj_num,1] = \
                                ((self.new_obj_tmp_Yolo_output[j,i+density_time*self.obj_num,1]*608).astype(int)+15)/608
                    if X_right > 607:
                        X_left = X_left-(X_right-607)
                        X_right = 607
                        self.new_obj_tmp_Yolo_output[j,i+density_time*self.obj_num,1] = \
                                ((self.new_obj_tmp_Yolo_output[j,i+density_time*self.obj_num,1]*608).astype(int)-15)/608
                    tmp_img[Y_down:Y_top,\
                            X_left:X_right,:] \
                    = self.obj_img[j,(self.new_obj_Class[j]+self.index*self.obj_num)[i+density_time*self.obj_num]]
            if j == 0:
                self.new_img = tmp_img.reshape(-1,608,608,3)
            else:
                self.new_img = np.append(self.new_img,tmp_img.reshape(-1,608,608,3),axis=0)
                
    def get_new_obj_Yolo_output(self):
        self.new_obj_Yolo_output = np.ones([self.new_img.shape[0],10,10,6])
        self.new_obj_Yolo_output[:,:,:,5] = np.multiply(self.new_obj_Yolo_output[:,:,:,5],0)
        for i in range(train_dataX.shape[0]):
            x = np.multiply(self.new_obj_tmp_Yolo_output[i,:,1].reshape(-1),10).astype(int)
            y = np.multiply(self.new_obj_tmp_Yolo_output[i,:,2].reshape(-1),10).astype(int)
            self.new_obj_Yolo_output[i,x,y,:] = self.new_obj_tmp_Yolo_output[i]


In [6]:
def DIQC(bbox1,bbox2):
    flag = 0
    bbox1_left = int((bbox1[0] - bbox1[2]/2) * 608)
    bbox1_right = int((bbox1[0] + bbox1[2]/2) * 608)
    bbox1_down = int((bbox1[1] - bbox1[3]/2) * 608)
    bbox1_top = int((bbox1[1] + bbox1[3]/2) * 608)
    
    bbox2_left = int((bbox2[0] - bbox2[2]/2) * 608)
    bbox2_right = int((bbox2[0] + bbox2[2]/2) * 608)
    bbox2_down = int((bbox2[1] - bbox2[3]/2) * 608)
    bbox2_top = int((bbox2[1] + bbox2[3]/2) * 608)
    
    
    if (bbox1_right > bbox2_left and bbox1_left < bbox2_right) or \
                    (bbox1_left < bbox2_right and bbox1_right > bbox2_left):
        flag = flag + 1
    if (bbox1_top > bbox2_down and bbox1_down < bbox2_top) or \
                        (bbox1_down < bbox2_top and bbox1_top > bbox2_down):
        flag = flag + 1
    
    if flag == 2:
        return 1
    else:
        return 0

In [7]:
def DESIMQC(yolo_loss_array):
    quilty_array = np.zeros([3])
    for i in np.arange(yolo_loss_array.shape[0]-1):
        quilty_array[i] = np.abs(yolo_loss_array[i] - yolo_loss_array[i+1])/yolo_loss_array[i]
    
    flag = 1
    for i in range(quilty_array.shape[0]):
        if quilty_array[i] > 0.5 or np.isnan(quilty_array[i]):
            flag = 0
    return quilty_array, flag
            

In [8]:
input_dim = 5*3*3
hidden1_dim = np.power(2,int(np.log2(input_dim)) + 5)
hidden2_dim = int(hidden1_dim / 4)
hidden3_dim = int(hidden2_dim / 4)
hidden4_dim = 45
hidden5_dim = hidden4_dim * 4
hidden6_dim = hidden5_dim * 4
output_dim = 10*10*(5+3)

print(hidden6_dim)
print(output_dim)


720
800


In [9]:
yolo_trainX = np.load('/home/louis/Documents/Python/Defect_Bean/Data/train_dataX.npy')
yolo_trainY = np.load('/home/louis/Documents/Python/Defect_Bean/Data/train_dataY_2.npy')

# train_dataX = train_dataX[35:64,:,:]
train_dataX = yolo_trainY[35:64-9,:]
yolo_trainX = yolo_trainX[35:64-9,:]
yolo_trainY = yolo_trainY[35:64-9,:]
print(train_dataX.shape)
print(yolo_trainX.shape)


(20, 10, 10, 6)
(20, 608, 608, 3)


In [10]:
tmp = np.asarray([])
for i in range(train_dataX.shape[0]):
    grid_x, grid_y = np.where(train_dataX[i,:,:,5] == 1)
    if i == 0:
        tmp = train_dataX[i,grid_x,grid_y,0:3].reshape(1,-1,3)
    else:
        tmp = np.append(tmp,train_dataX[i,grid_x,grid_y,0:3].reshape(1,-1,3),axis=0)
print(tmp.shape)
generate_input = tmp
img_obj_info = tmp

(20, 5, 3)


In [11]:
generate_input = generate_input.reshape(generate_input.shape[0],-1)
print(generate_input.shape)
generate_input = np.append(generate_input,np.zeros([generate_input.shape[0],30]),axis=1)
print(generate_input.shape)

img = yolo_trainX
label = yolo_trainY
print(img.shape)
print(label.shape)

(20, 15)
(20, 45)
(20, 608, 608, 3)
(20, 10, 10, 6)


In [12]:
learning_rate = 0.0001
training_epochs = 1000
GA_epochs = 50
batch_size = 20

In [13]:
yolo = YoloModel('/home/louis/Documents/Python/Defect_Bean/Yolo_Model/Test/model_noNorm.meta')
ga = GA(yolo)

INFO:tensorflow:Restoring parameters from /home/louis/Documents/Python/Defect_Bean/Yolo_Model/Test/model_noNorm.meta


In [14]:
print(label.shape)
a = (label[:,:,:,0]).astype(np.int)
n_values = 14
a = np.eye(n_values)[a]
yolo_train_dataY = np.append(a,label[:,:,:,1:],axis=3).reshape(-1,10*10,19)
print(yolo_train_dataY.shape)
yolo_train_dataX = img / 255


(20, 10, 10, 6)
(20, 100, 19)


In [15]:
k = 0
yolo_loss_array = np.zeros([4])

In [ ]:
while (1):
#     batch_size = yolo_train_dataX.shape[0]
    print('k ',k)
    if k == 0:
        yolo.optimize(yolo_train_dataX,yolo_train_dataY,learning_rate,training_epochs,batch_size)
        yolo.run(yolo_train_dataX,yolo_train_dataY,batch_size)
        yolo_loss_array[0] = yolo.loss
        yolo.save('/home/louis/Documents/Python/Defect_Bean/Yolo_Model/Main/model_noNorm_'+str(k)+'.meta')
    else :
        yolo.optimize(yolo_train_dataX,yolo_train_dataY,learning_rate,training_epochs,batch_size)
        yolo.save('/home/louis/Documents/Python/Defect_Bean/Yolo_Model/Main/model_noNorm_'+str(k)+'.meta')

    ga.set_gener_model()

    for i in range(GA_epochs):
        if i % 10 == 0 and i != 0:
            print('GA_epochs ',i)
        ga.run_gener_model(generate_input,yolo_trainX,yolo_trainY)
        ga.get_adaptability_probability()
        ga.run_gener_model_weight()
        ga.run_gener_model_bias()
        ga.change_mode()
    
    generator = ga.get_target_model()
    gener_LBIG = LBIG()
    gener_LBIG.set_model(generator.gener_W,generator.gener_B)

    gener_data_input = np.zeros([3,yolo_trainY.shape[0],45])
    gener_data_img = np.zeros([3,yolo_trainX.shape[0],3,608,608,3])
    obj_info = np.zeros([3,yolo_trainY.shape[0],3,10,10,6])

    for density in range(3):
        print('density ',density)
        for gener_times in range(img_obj_info.shape[0]):
            for i in range(density+1):
                choose_picture_info = np.random.randint(img_obj_info.shape[0])
                tmp = img_obj_info[choose_picture_info]
                gener_data_input[density,gener_times,i*15:(i+1)*15] = tmp.reshape(-1)
                gener_data_img[density,gener_times,i] = yolo_trainX[choose_picture_info]
                obj_info[density,gener_times,i] = yolo_trainY[choose_picture_info]

    generate_img_data = []
    generate_yolo_output_data = []
    for density in range(3):
        gener_LBIG.run(gener_data_input[density])
        gener_LBIG.get_new_obj_XY_Class(density+1)
        gener_LBIG.get_new_obj_tmp_Yolo_output(obj_info[density,:],density+1)
        gener_LBIG.get_obj_img(gener_data_img[density,:],obj_info[density,:],density+1)
        gener_LBIG.get_new_img(density+1)
        gener_LBIG.get_new_obj_Yolo_output()

        if density == 0:
            generate_img_data = gener_LBIG.new_img
            generate_yolo_output_data = gener_LBIG.new_obj_Yolo_output
        else:
            generate_img_data = np.append(generate_img_data,gener_LBIG.new_img,axis=0)
            generate_yolo_output_data = np.append(generate_yolo_output_data,gener_LBIG.new_obj_Yolo_output,axis=0)
    print(generate_yolo_output_data.shape)

    img = np.append(img,generate_img_data,axis=0)
    label = np.append(label,generate_yolo_output_data,axis=0)

#     generate_input = np.asarray([])
#     for i in range(label.shape[0]):
#         grid_x, grid_y = np.where(label[i,:,:,5] == 1)
#         if i == 0:
#             generate_input = np.append(label[i,grid_x,grid_y,0:3].reshape(1,-1),np.zeros([1,30]),axis=1)
#         else:
#             if (label[i,grid_x,grid_y,0:3].reshape(1,-1).shape[1]<45):
#                 generate_input = np.append(generate_input,\
#                                              np.append(label[i,grid_x,grid_y,0:3].reshape(1,-1),\
#                                                        np.zeros([1,45-(label[i,grid_x,grid_y,0:3].reshape(1,-1).shape[1])])
#                                                        ,axis=1),axis=0)
#             else:
#                 generate_input = np.append(generate_input,label[i,grid_x,grid_y,0:3].reshape(1,-1),axis=0)

#     print(generate_input.shape)

    quality_index = []
    for label_index in range(label.shape[0]):
        overLapping = 0
        yolo_output = label[label_index]
        first_dim,second_dim = np.where(yolo_output[:,:,5]==1)
        obj_info = yolo_output[first_dim,second_dim,:]
        for i in range(obj_info.shape[0]):
            for j in range(obj_info.shape[0]-(i+1)):
                j = j+i+1
                overLapping = overLapping + DIQC(obj_info[i,1:5],obj_info[j,1:5])
        if  overLapping == 0:
            quality_index.append(label_index)
    print('quality_index ',quality_index)

    img = img[quality_index]
    label = label[quality_index]
#     generate_input = generate_input[quality_index]
    
    
    print('label.shape ',label.shape)
    a = (label[:,:,:,0]).astype(np.int)
    n_values = 14
    a = np.eye(n_values)[a]
    yolo_train_dataY = np.append(a,label[:,:,:,1:],axis=3).reshape(-1,10*10,19)
    print('yolo_train_dataY.shape ',yolo_train_dataY.shape)
    yolo_train_dataX = img / 255


    yolo.run(yolo_train_dataX,yolo_train_dataY,batch_size) 
    yolo_loss_array[(k)%4] = yolo.loss

    if k >= 4:
        quailty_array,flag = DESIMQC(yolo_loss_array)
        print('quailty_array ',quailty_array)
        if flag:
            break
            
    print('yolo_loss_array ',yolo_loss_array)
#     print(quailty_array)

    k = k+1
    np.save('Data/LBIG_img_' + str(k),img)
    np.save('Data/LBIG_L_' + str(k),label)
    np.save('Data/LBIG_generate_input_' + str(k),generate_input)

k  0
batch_num :  1
epoch :  200
batch_loss 81752728.0
epoch :  400
batch_loss 76243704.0
epoch :  600
batch_loss 73546776.0
epoch :  800
batch_loss 72193080.0
epoch :  1000
batch_loss 71500720.0
epoch :  1200
batch_loss 71134552.0
epoch :  1400
batch_loss 70955616.0
epoch :  1600
batch_loss 70863856.0
epoch :  1800
batch_loss 70819368.0
calculate_Adaptability_done
run_gener_model_weight_done
run_gener_model_bias_done
calculate_Adaptability_done
run_gener_model_weight_done
run_gener_model_bias_done
calculate_Adaptability_done
run_gener_model_weight_done
run_gener_model_bias_done
calculate_Adaptability_done
run_gener_model_weight_done
run_gener_model_bias_done
calculate_Adaptability_done
run_gener_model_weight_done
run_gener_model_bias_done
calculate_Adaptability_done
run_gener_model_weight_done
run_gener_model_bias_done
calculate_Adaptability_done
run_gener_model_weight_done
run_gener_model_bias_done
calculate_Adaptability_done
run_gener_model_weight_done
run_gener_model_bias_done
calc

calculate_Adaptability_done
run_gener_model_weight_done
run_gener_model_bias_done
calculate_Adaptability_done
run_gener_model_weight_done
run_gener_model_bias_done
calculate_Adaptability_done
run_gener_model_weight_done
run_gener_model_bias_done
calculate_Adaptability_done
run_gener_model_weight_done
run_gener_model_bias_done
calculate_Adaptability_done
run_gener_model_weight_done
run_gener_model_bias_done
calculate_Adaptability_done
run_gener_model_weight_done
run_gener_model_bias_done
calculate_Adaptability_done
run_gener_model_weight_done
run_gener_model_bias_done
GA_epochs  40
calculate_Adaptability_done
run_gener_model_weight_done
run_gener_model_bias_done
calculate_Adaptability_done
run_gener_model_weight_done
run_gener_model_bias_done
calculate_Adaptability_done
run_gener_model_weight_done
run_gener_model_bias_done
calculate_Adaptability_done
run_gener_model_weight_done
run_gener_model_bias_done
calculate_Adaptability_done
run_gener_model_weight_done
run_gener_model_bias_done
ca

In [ ]:
# tmp_yolo = YoloModel('/home/louis/Documents/Python/Defect_Bean/Yolo_Model/Main/model_noNorm_part2.meta')

In [ ]:
# tmp_yolo.run(yolo_train_dataX,yolo_train_dataY)
# tmp_yolo.loss

In [ ]:
# tmp_yolo_train_dataX = yolo_train_dataX
# tmp_yolo_train_dataY = yolo_train_dataY

In [ ]:
# tmp_yolo.optimize(yolo_train_dataX,yolo_train_dataY,0.00001,training_epochs,batch_size)

In [ ]:
# tmp_yolo.pre